In [1]:
import tensorflow as tf

hello = tf.constant('Hello, Tensorflow')
print(hello)

a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a, b)
print(c)

sess = tf.Session()
print(sess.run(hello))
print(sess.run([a, b, c]))

sess.close()


Tensor("Const:0", shape=(), dtype=string)
Tensor("Add:0", shape=(), dtype=int32)
b'Hello, Tensorflow'
[10, 32, 42]


### Variable

In [3]:
import tensorflow as tf

# tf.placeholder: 계산을 실행할 때 입력값을 받는 변수로 사용합니다.
# None은 크기가 정해지지 않았음을 의미합니다.
X = tf.placeholder(tf.float32, [None, 3])
print(X)

x_data = [[1, 2, 3], [4, 5, 6]]

# tf.random_normal: 각 변수들의 초기값을 정규분포 랜덤 값으로 초기화합니다.
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

expr = tf.matmul(X, W) + b

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print(x_data)
print(sess.run(W))
print(sess.run(b))
print(sess.run(expr, feed_dict={X: x_data}))

sess.close()

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)
[[1, 2, 3], [4, 5, 6]]
[[ 0.95473993 -0.916623  ]
 [ 1.07248819 -0.05501784]
 [-0.31067339  0.35213915]]
[[-0.74425435]
 [ 2.31388235]]
[[ 1.42344165 -0.71449554]
 [ 9.63124275  0.48513603]]


### MNIST

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("C:/Users/SEC/Desktop/MNIST_data/", one_hot=True)

import tensorflow as tf

print(tf.convert_to_tensor(mnist.train.images).get_shape())
print(tf.convert_to_tensor(mnist.train.labels).get_shape())

# 784 input units, 10 hidden units
W = tf.Variable(tf.zeros([784, 10]))
# 10 biases attached to hidden units
b = tf.Variable(tf.zeros([10]))
x = tf.placeholder("float", [None, 784])
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_= tf.placeholder("float", [None, 10])
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
sess = tf.Session()
sess.run(tf.global_variables_initializer())


for step in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict = {x:batch_xs, y_:batch_ys}) # feed x and y_ to train_step

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,  1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    accuracy_ = sess.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels})
    print('step:{:01d} | accuracy : {:.4f}'.format(step, float(accuracy_)))
    

Extracting C:/Users/SEC/Desktop/MNIST_data/train-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data/train-labels-idx1-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 10)
step:0 | accuracy : 0.3665
step:1 | accuracy : 0.4098
step:2 | accuracy : 0.3831
step:3 | accuracy : 0.4102
step:4 | accuracy : 0.4647
step:5 | accuracy : 0.5186
step:6 | accuracy : 0.5745
step:7 | accuracy : 0.7156
step:8 | accuracy : 0.6764
step:9 | accuracy : 0.7625
step:10 | accuracy : 0.7512
step:11 | accuracy : 0.7581
step:12 | accuracy : 0.7069
step:13 | accuracy : 0.7576
step:14 | accuracy : 0.7260
step:15 | accuracy : 0.7993
step:16 | accuracy : 0.7875
step:17 | accuracy : 0.7940
step:18 | accuracy : 0.7742
step:19 | accuracy : 0.7563
step:20 | accuracy : 0.7699
step:21 | accuracy : 0.8277
step:22 | accuracy : 0.8574


KeyboardInterrupt: 

### Implementing the Regression

In [8]:
'''
To do efficient numerical computing in Python, we typically use libraries like NumPy that
do expensive operations such as matrix multiplication outside Python, using highly 
efficient code implemented in another language. 

Unfortunately, there can still be a lot of overhead from switching back to Python every operation.
This overhead is especially bad if you want to run computations on GPUs or in a distributed manner,
where there can be a high cost to transferring data.

Instead of running a single expensive operation independently from Python, Tensorflow 
lets us describe a graph of interacting operations that run entirely outside Python.
'''
import tensorflow as tf

# we describe these interacting opearations by manipulating symbolic variables.
x = tf.placeholder(tf.float32, [None, 784])

'''
x isn't a specific value. It's a placeholder, a value that we'll input when we ask
TensorFlow to run a computation. We want to be able to input any number of MNIST images,
each flattened into a 784-dimensional vector. We represent this as a 2-D tensor of floating-point
numbers, with a shape [None, 784]. (Here None means that a dimension can be of any length)

We also need the weights and biases for our model. We could imagine treating these like 
additional inputs, but Tensorflow has an even better way to handle it: Variable. A Variable
is a modifiable tensor that lives in TensorFlow's graph of interacting operations.
'''

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

'''
We create these Variables by giving tf.Variable the initial value of Variable:
in this case, we initialize both W and b as tensors full of zeros. Since we are
going to learn W and b, it doesn't matter very much what they initially are.

Notice that W has a shape of [784, 10] because we want to multiply the 784-dimensional
image vectors by it to produce 10-dimensional vectors of evidence for the difference
classes. b has a shape of [10] so we can add it to the output.

We can now implement our model. It only takes one line to define it!
'''

y = tf.nn.softmax(tf.matmul(x, W) + b)

'''
First, we multiply x by W with the expression tf.matmul(x, W). This is flipped from when
we multiplied them in our equation, where we had Wx, as a small trick to deal with x being
a 2D tensor with multiple inptu.s We then add b, and finally apply tf.nn.softmax.

That's it. It only took us one line to define our model, after a couple short lines of setup.

That isn't because Tensorflow is designed to make a softmax regression particularly 
easy: it's just a very flexible way to describe many kinds of numerical computations,
from machine learning models to physics simulations. And once defined, our model can be run
on different devices: your computer's CPU, GPUs, and even phones!
'''

"\nFirst, we multiply x by W with the expression tf.matmul(x, W). This is flipped from when\nwe multiplied them in our equation, where we had Wx, as a small trick to deal with x being\na 2D tensor with multiple inptu.s We then add b, and finally apply tf.nn.softmax.\n\nThat's it. It only took us one line to define our model, after a couple short lines of setup.\n\nThat isn't because Tensorflow is designed to make a softmax regression particularly \neasy: it's just a very flexible way to describe many kinds of numerical computations,\nfrom machine learning models to physics simulations. And once defined, our model can be run\non different devices: your computer's CPU, GPUs, and even phones!\n"

### Cross Entropy

In [6]:
'''
One very common, very nice function to determine the loss of a model is called
"cross-entropy." Cross-entropy arises from thinking about information compressing codes
in information theory, but it winds up being an important idea in lots of areas, from gambling
to machine learning. It's defined as:

H(y) = - sigma(y' * log(y))

where y is our predicted probability distribution, and y' is the true distribution (the one-hot
vector with the digit labels.) In some rough sense, the cross-entropy is measuring how inefficient
our predictions are for describing the truth.

To implement cross-entropy we need to first add a new placeholder to input the correct answers:
'''
y_ = tf.placeholder(tf.float32, [None, 10])

# Then we can implement the cross-entropy function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

'''
First, tf.log computes the logarithm of each element of y. Next, we multiply each element of y_
with the corresponding element of tf.log(y). Then, tf.reduce_sum adds the elements in the
second dimension of y, due to the reduction_indices=[1] parameter. Finally, tf.reduce_mean
computes the mean over all the examples in the batch.

Note that in the source code, we don't use this formulation, because it is numerically 
unstable. Instead, we apply tf.nn.softmax_cross_entropy_with_logits on the unnormalized
logits (e.g., we call softmax_cross_entropy_with_logits on tf.matmul(x, W) + b), because
this more numerically stable function internally computes the softmax activation. In your 
code, consider using tf.nn.sofmax_cross_entropy_with_logits instead.

Now that we know what we want our model to do, it's very easy to have TensorFlow train
it to do so. Because TensorFlow knows the entire graph of your computations, it can automatically
use the backpropagation algorithm to efficiently determine how your variables affect
the loss you ask it to minimize. Then it can apply your choice of optimization algorithm
to modify the variables and reduce the loss
'''
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

'''
In this case, we ask Tensorflow to minimize cross_entropy using the graident descent algorithm
with a learning rate of 0.5.
'''

sess = tf.InteractiveSession()
# we first have to create an operation to initialize tha variables we created
tf.global_variables_initializer().run()


for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
    


In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("C:/Users/SEC/Desktop/MNIST_data", one_hot=True)

import tensorflow as tf

print(tf.convert_to_tensor(mnist.train.images).get_shape())
print(tf.convert_to_tensor(mnist.train.labels).get_shape())

# set variables
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# placeholder for input data
x = tf.placeholder("float", [None, 784])
y_ = tf.placeholder("float", [None, 10])

# placeholder for learning rate
learning_rate = tf.placeholder("float", shape=[])

# build softmax model
y = tf.nn.softmax(tf.matmul(x,W) + b)

# loss function
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# optimization
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

#set init learning rate
lr = 0.01

for step in range(1000):

    if(step % 100 == 0 and step != 0):
        lr = lr/10

    # get 100 images for each iteration
    batch_xs, batch_ys = mnist.train.next_batch(100)

    # train the model
    sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys, learning_rate:lr})

    # test teh results
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    accuracy_ = sess.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels})

    print('step: {:01d} | accuracy : {:.4f} | learning rate : {:.4f}'.format(step, float(accuracy_), lr))


Extracting C:/Users/SEC/Desktop/MNIST_data\train-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 10)
step: 0 | accuracy : 0.2808 | learning rate : 0.0100
step: 1 | accuracy : 0.5905 | learning rate : 0.0100
step: 2 | accuracy : 0.4665 | learning rate : 0.0100
step: 3 | accuracy : 0.4844 | learning rate : 0.0100
step: 4 | accuracy : 0.3135 | learning rate : 0.0100
step: 5 | accuracy : 0.5828 | learning rate : 0.0100
step: 6 | accuracy : 0.5399 | learning rate : 0.0100
step: 7 | accuracy : 0.6283 | learning rate : 0.0100
step: 8 | accuracy : 0.5893 | learning rate : 0.0100
step: 9 | accuracy : 0.6929 | learning rate : 0.0100
step: 10 | accuracy : 0.7203 | learning rate : 0.0100
step: 11 | accuracy : 0.6702 | learning rate : 0.0100
step: 12 | accuracy : 0.6963 | learning rate : 0.0100
step:

KeyboardInterrupt: 

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("C:/Users/SEC/Desktop/MNIST_data", one_hot=True)

import tensorflow as tf

print(tf.convert_to_tensor(mnist.train.images).get_shape())
print(tf.convert_to_tensor(mnist.train.labels).get_shape())

# set variables
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# placeholder for input data
x = tf.placeholder("float", [None, 784])
y_ = tf.placeholder("float", [None, 10])

# placeholder for learning rate
learning_rate = tf.placeholder("float", shape=[])

# build softmax model
y = tf.nn.softmax(tf.matmul(x,W) + b)

# loss function
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# optimization
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

#set init learning rate
lr = 0.01

# step decay, exponential decay, 1/t decay
for step in range(1000):

    if(step % 100 == 0 and step != 0):
        lr = lr / 10

    # get 100 images for each iteration
    batch_xs, batch_ys = mnist.train.next_batch(100)

    # train the model
    sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys, learning_rate:lr})

    # test teh results
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    accuracy_ = sess.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels})

    print('step: {:01d} | accuracy : {:.4f} | learning rate : {:.4f}'.format(step, float(accuracy_), lr))


Extracting C:/Users/SEC/Desktop/MNIST_data\train-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 10)
step: 0 | accuracy : 0.2106 | learning rate : 0.0100
step: 1 | accuracy : 0.4193 | learning rate : 0.0100
step: 2 | accuracy : 0.3233 | learning rate : 0.0100
step: 3 | accuracy : 0.5013 | learning rate : 0.0100
step: 4 | accuracy : 0.4797 | learning rate : 0.0100
step: 5 | accuracy : 0.5484 | learning rate : 0.0100
step: 6 | accuracy : 0.5543 | learning rate : 0.0100
step: 7 | accuracy : 0.6362 | learning rate : 0.0100
step: 8 | accuracy : 0.5875 | learning rate : 0.0100
step: 9 | accuracy : 0.6278 | learning rate : 0.0100
step: 10 | accuracy : 0.6399 | learning rate : 0.0100
step: 11 | accuracy : 0.6881 | learning rate : 0.0100
step: 12 | accuracy : 0.7412 | learning rate : 0.0100
step:

KeyboardInterrupt: 

In [ ]:
'''
Batch Size

 네트워크를 한번 propagation할 때 데이터 수를 의미
 batch size = 보통 128개
 ex. 1050개의 데이터, batch size 100
  - 처음 포워드할 때 [1~100], 다음 [101~200], [201~300], ..., [1000~1050]
  - 데이터를 한번 다 도는 것을 1 Epoch라 함.

'''

In [ ]:
# MLP

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("C:/Users/SEC/Desktop/MNIST_data", one_hot=True)

import tensorflow as tf

print(tf.convert_to_tensor(mnist.train.images).get_shape())
print(tf.convert_to_tensor(mnist.train.labels).get_shape())

a_0 = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

middle = 30
w_1 = tf.Variable(tf.truncated_normal([784, middle]))
b_1 = tf.Variable(tf.truncated_normal([1, middle]))
w_2 = tf.Variable(tf.truncated_normal([middle, 10]))
b_2 = tf.Variable(tf.truncated_normal([1, 10]))

def sigma(x):
    return tf.div(tf.constant(1.0),
                  tf.add(tf.constant(1.0), tf.exp(tf.negative(x))))
def sigmaprime(x):
    return tf.multiply(sigma(x), tf.subtract(tf.constant(1.0), sigma(x)))


z_1 = tf.add(tf.matmul(a_0, w_1), b_1)
a_1 = sigma(z_1)
z_2 = tf.add(tf.matmul(a_1, w_2), b_2)
a_2 = sigma(z_2)

diff = tf.subtract(a_2, y)
d_z_2 = tf.multiply(diff, sigmaprime(z_2))
d_b_2 = d_z_2
d_w_2 = tf.matmul(tf.transpose(a_1), d_z_2)

d_a_1 = tf.matmul(d_z_2, tf.transpose(w_2))
d_z_1 = tf.multiply(d_a_1, sigmaprime(z_1))
d_b_1 = d_z_1
d_w_1 = tf.matmul(tf.transpose(a_0), d_z_1)

eta = tf.constant(0.5)
step = [
    tf.assign(w_1,
            tf.subtract(w_1, tf.multiply(eta, d_w_1)))
  , tf.assign(b_1,
            tf.subtract(b_1, tf.multiply(eta,
                               tf.reduce_mean(d_b_1, axis=[0]))))
  , tf.assign(w_2,
            tf.subtract(w_2, tf.multiply(eta, d_w_2)))
  , tf.assign(b_2,
            tf.subtract(b_2, tf.multiply(eta,
                               tf.reduce_mean(d_b_2, axis=[0]))))
]

acct_mat = tf.equal(tf.argmax(a_2, 1), tf.argmax(y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in xrange(10000):
    batch_xs, batch_ys = mnist.train.next_batch(10)
    sess.run(step, feed_dict = {a_0: batch_xs,
                                y : batch_ys})
    if i % 1000 == 0:
        res = sess.run(acct_res, feed_dict =
                       {a_0: mnist.test.images[:1000],
                        y : mnist.test.labels[:1000]})
        print res



In [7]:
# A Multilayer Perceptron implementation example using TensorFlow

from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("C:/Users/SEC/Desktop/MNIST_data", one_hot=True)

import tensorflow as tf

print(tf.convert_to_tensor(mnist.train.images).get_shape())
print(tf.convert_to_tensor(mnist.train.labels).get_shape())

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

    

Extracting C:/Users/SEC/Desktop/MNIST_data\train-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 10)
Epoch: 0001 cost= 166.349192429
Epoch: 0002 cost= 38.444730281
Epoch: 0003 cost= 24.216119050
Epoch: 0004 cost= 16.946142122
Epoch: 0005 cost= 12.332160136
Epoch: 0006 cost= 9.256018394
Epoch: 0007 cost= 7.025145741
Epoch: 0008 cost= 5.209423389
Epoch: 0009 cost= 4.002566180
Epoch: 0010 cost= 2.987386289
Epoch: 0011 cost= 2.248583235
Epoch: 0012 cost= 1.697362977
Epoch: 0013 cost= 1.322869226
Epoch: 0014 cost= 1.061509206
Epoch: 0015 cost= 0.810548451
Optimization Finished!
Accuracy: 0.9457


In [1]:
import tensorflow as tf

x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

add = tf.add(x, y)
mul = tf.multiply(x, y)

add_hist = tf.summary.scalar("add_scalar", add)
mul_hist = tf.summary.scalar("mul_scalar", mul)

merged = tf.summary.merge_all()

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    writer = tf.summary.FileWriter("./sample", sess.graph)
    
    for step in range(100):
        summary = sess.run(merged, feed_dict={x: step * 1.0, y : 2.0})
        writer.add_summary(summary, step)
        


In [5]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("C:/Users/SEC/Desktop/MNIST_data", one_hot=True)

import tensorflow as tf

print(tf.convert_to_tensor(mnist.train.images).get_shape())
print(tf.convert_to_tensor(mnist.train.labels).get_shape())

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

x = tf.placeholder("float", [None, 784])
y_ = tf.placeholder("float", [None, 10])

y = tf.nn.softmax( tf.matmul(x,W) + b )

cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# choose node
loss_hist = tf.summary.scalar("cross_entropy", cross_entropy)
accur_hist = tf.summary.scalar("valid_hist", accuracy)

# merge all
merged = tf.summary.merge_all()

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
sess = tf.Session()
sess.run(tf.initialize_all_variables())

# set writer of train and test
train_writer = tf.summary.FileWriter("./sample/train", sess.graph)
test_writer = tf.summary.FileWriter("./sample/test", sess.graph)

for step in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    
    # run loss_hist and train_step
    summary, _ = sess.run([loss_hist, train_step], feed_dict={x: batch_xs, y_: batch_ys})
    
    # add summary of loss hist
    train_writer.add_summary(summary, step)

    #run accur_hist and accuracy
    summary, accuracy_ = sess.run([accur_hist, accuracy], feed_dict={x:mnist.test.images,
                                        y_:mnist.test.labels})
    # add summary of accuracy hist
    test_writer.add_summary(summary, step)

    print('step: {:01d} | accuracy : {:.4f}'.format(step,float(accuracy_)))
    



Extracting C:/Users/SEC/Desktop/MNIST_data\train-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 10)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
step: 0 | accuracy : 0.3478
step: 1 | accuracy : 0.2854
step: 2 | accuracy : 0.3315
step: 3 | accuracy : 0.5152
step: 4 | accuracy : 0.4529
step: 5 | accuracy : 0.5188
step: 6 | accuracy : 0.5647
step: 7 | accuracy : 0.6730
step: 8 | accuracy : 0.7221
step: 9 | accuracy : 0.7390
step: 10 | accuracy : 0.6830
step: 11 | accuracy : 0.7413
step: 12 | accuracy : 0.6395
step: 13 | accuracy : 0.6661
step: 14 | accuracy : 0.7462
step: 15 | accuracy : 0.7428
step: 16 | accuracy : 0.6805
step: 17 | accuracy : 0.7737
step: 18 | accuracy : 0.8471
step: 19 | accuracy : 0.7987
step: 20 | accuracy : 0.8411
step: 21 | accuracy : 0.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("C:/Users/SEC/Desktop/MNIST_data", one_hot=True)

import tensorflow as tf

print(tf.convert_to_tensor(mnist.train.images).get_shape())
print(tf.convert_to_tensor(mnist.train.labels).get_shape())

# You can freeze your layer by setting trainable=False
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

x = tf.placeholder("float", [None, 784])
y_ = tf.placeholder("float", [None, 10])

learning_rate = tf.placeholder('float', [])

y = tf.nn.softmax( tf.matmul(x,W) + b )

cross_entropy = -tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver({"W": W})

lr = 0.01
for step in range(200):
    
    # saver.restore(sess, './model/model_100.ckpt')
    
    if(step % 100 == 0 and step != 0):
        save_path = "./model/model_" + str(step) + '.ckpt'
        saver.save(sess, save_path)
        lr = lr / 10
    
    batch_xs, batch_ys = mnist.train.next_batch(100)
    
    # run loss_hist and train_step
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, lr: learning_rate})
    
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    accuracy_ = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})

    print('step: {:01d} | accuracy : {:.4f}'.format(step,float(accuracy_)))
    



Extracting C:/Users/SEC/Desktop/MNIST_data\train-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:/Users/SEC/Desktop/MNIST_data\t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 10)
